In [ ]:
#目标编码
#聚合统计特征
#TFIDF
#w2v. 用户属性平均。
#w2v。 根据统计的权重


In [ ]:
from gensim.models import Word2Vec
import pandas as pd
import numpy as np
import os

In [ ]:
train_root_path ='./data/train_preliminary/'
train_ad_path = os.path.join(train_root_path,'ad.csv')
train_click_path = os.path.join(train_root_path,'click_log.csv')
train_user_path = os.path.join(train_root_path,'user.csv')

In [ ]:
test_root_path = './data/test/'
test_ad_path = os.path.join(test_root_path,'ad.csv')
test_click_path = os.path.join(test_root_path,'click_log.csv')

In [ ]:
df_ad_train = pd.read_csv(train_ad_path,na_values='\\N', 
                          dtype= {'creative_id':str,'ad_id': str, 'product_id': str, 'product_category': str,'advertiser_id': str,'industry': str} )
df_ad_test = pd.read_csv(test_ad_path,na_values='\\N',
                         dtype= {'creative_id':str,'ad_id': str, 'product_id': str, 'product_category': str,'advertiser_id': str,'industry': str} )

In [ ]:
df_click_train = pd.read_csv(train_click_path,dtype={'creative_id':str,'click_times':str,'time':str})
df_click_test = pd.read_csv(test_click_path,dtype={'creative_id':str,'click_times':str,'time':str})

In [ ]:
df_click_ad_train = df_click_train.merge(df_ad_train,on=['creative_id'],how='inner')
df_click_ad_test = df_click_test.merge(df_ad_test,on=['creative_id'],how='inner')

In [ ]:
df_click_all =  pd.concat([df_click_ad_train,df_click_ad_test], axis=0)
df_click_all = df_click_all.sort_values('time', ascending=True)

In [ ]:
df_user = pd.read_csv(train_user_path)

In [ ]:
df_click_all.fillna('un',inplace=True)

In [ ]:
df_click_all

# 数据处理

In [ ]:
df_click_all = df_click_all.reset_index()

In [ ]:
aggcol = ['creative_id','ad_id','product_id','product_category','advertiser_id','industry','time','click_times']

In [ ]:
func = lambda x: ' '.join(x.tolist())
agg_funcs ={}
for col in aggcol:
    agg_funcs[col] = func
group_df = df_click_all.groupby(['user_id']).agg(agg_funcs)

In [ ]:
group_df = group_df.reset_index()

In [ ]:
df_data = group_df.merge(df_user, on='user_id', how='left')

In [ ]:
df_data.to_parquet('./data/cache/df_data.parquet',index=False)

In [ ]:

#加载缓存
df_data =pd.read_parquet('./data/cache/df_data.parquet')

In [ ]:
df_data

# tfidf 

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from tqdm import tqdm


In [ ]:
df_test_final = df_data[df_data.age.isna()]
df_train_val_final = df_data[~df_data.age.isna()]

In [ ]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# tfidf = TfidfVectorizer(min_df=30,max_features=100000)
# tfidf.fit(group_df['creative_id'])

In [ ]:
def get_sklearn_classfiy_stacking(tfname,clf, train_feature, test_feature, score, model_name, class_number, n_folds, train_num, test_num):
    print('\n****开始跑', model_name, '****')
    stack_train = np.zeros((train_num, class_number))
    stack_test = np.zeros((test_num, class_number))
    score_mean = []
    skf = StratifiedKFold(n_splits=n_folds, random_state=1017)
    for i, (tr, va) in enumerate(skf.split(train_feature, score)):
        clf.fit(train_feature[tr], score[tr])
        score_va = clf._predict_proba_lr(train_feature[va])
        score_te = clf._predict_proba_lr(test_feature)
        score_single = accuracy_score(score[va], np.argmax(score_va, axis=1))
        score_mean.append(np.around(score_single, 5))
        print(f'{i+1}/{n_folds}', score_single)
        stack_train[va] += score_va
        stack_test += score_te
    stack_test /= n_folds
    stack = np.vstack([stack_train, stack_test])
    df_stack = pd.DataFrame()
    for i in range(stack.shape[1]):
        df_stack[f'{tfname}_{model_name}_{str(i)}'] = stack[:, i]
    print(model_name, '处理完毕')
    return df_stack, score_mean

model_list = [
    ['LogisticRegression', LogisticRegression(random_state=1017, C=3)],
    ['SGDClassifier', SGDClassifier(random_state=1017, loss='log')],
    ['PassiveAggressiveClassifier', PassiveAggressiveClassifier(random_state=1017, C=2)],
    ['RidgeClassfiy', RidgeClassifier(random_state=1017)],
    ['LinearSVC', LinearSVC(random_state=1017)]
]

In [ ]:
for col in aggcol:
    print(f'开始计算tf-idf特征:{col}')
    tf = TfidfVectorizer(min_df=30,max_features=100000,ngram_range=(1, 1))
    discuss_tf = tf.fit_transform(df_data[col]).tocsr()
    print('计算结束')

    ############################ 切分数据集 ##########################
    print('开始进行一些前期处理')
    train_feature = discuss_tf[:len(df_train_val_final)]
    score = df_train_val_final['age'] - 1
    test_feature = discuss_tf[len(df_train_val_final):]
    print('处理完毕')

    ######################### 模型函数(返回sklean_stacking结果) ########################


    feature = pd.DataFrame()
    for i in model_list:
        stack_result, score_mean = get_sklearn_classfiy_stacking(col,i[1], train_feature, test_feature, score, i[0], 10, 5, len(df_train_val_final), len(df_test_final))
        feature = pd.concat([feature, stack_result], axis=1, sort=False)
        print('五折结果', score_mean)
        print('平均结果', np.mean(score_mean))
    feature.to_csv(f'./model/cache/{col}_tfidf.csv', index=False)

# 生成用户的embeding

In [ ]:
aggcol_without_time = [t for t in aggcol if t not in ['time','click_times'] ]

In [ ]:
aggcol_without_time

In [ ]:
from gensim.models import KeyedVectors
wv_group ={}
for col in aggcol_without_time:
    wv_group[col] = KeyedVectors.load(f"./model/word2vec_{col}_128.model", mmap='r')

In [ ]:
for col_name in aggcol_without_time:
    def agg_emebeding(x):
        items =  x.split(' ')
        user_embed =  np.zeros((128))
        wlen = 0
        for k in items:
            if k == 'un':
                continue
            user_embed+=wv_group[col_name][k]
            wlen +=1
        
        user_embed = user_embed/wlen
        return user_embed
    df_data[col_name+'_user_embed'] = df_data[col_name].apply(agg_emebeding)

In [ ]:
df_data[[col_name+'_user_embed' for col_name in aggcol_without_time]].to_parquet('./model/cache/user_embed.parquet',index=False)

In [ ]:
df_data[['creative_id_user_embed']].to_parquet('./model/cache/user_embed_creative_id.parquet',index=False)